### 1.- Groupby
A continuación se muestran algunos ejemplos sobre el uso de `Groupby`

In [1]:
import pandas as pd
import numpy as np

# Genero dataframe de ejemplo:
df_test = pd.DataFrame({
    'class':["A","A","B","B","B"],
    'item_id':[1,2,1,3,3,],  
    'ml':[100,150,100,180,180],
    'price':[10,15,10,14,14]
})

df_test

,class,item_id,ml,price
0,A,1,100,10
1,A,2,150,15
2,B,1,100,10
3,B,3,180,14
4,B,3,180,14


Agrupo por "class" y visualizo cuántos elementos hay en cada grupo:

In [2]:
# Tamaño de cada grupo - 1:
df_test.groupby("class").size()

class
A    2
B    3
dtype: int64

In [3]:
# Tamaño de cada grupo - 2:
df_test.groupby("class").size().to_frame()

,0
class,
A,2
B,3


In [4]:
# Tamaño de cada grupo - 3:
df_test.groupby("class").size().to_frame().reset_index()

,class,0
0,A,2
1,B,3


Con `.to_frame()` y `.reset_index()`pudimos cambiar la salida del `groupby().size()`

Tambien podemos tomar los elementos de un grupo en particular:

In [5]:
# Asgino los grupos a otro DF:
df_test_grp = df_test.groupby("class")
df_test_grp.size()

class
A    2
B    3
dtype: int64

In [6]:
# Obtengo los elementos del grupo B
df_test_grp.get_group('B')

,class,item_id,ml,price
2,B,1,100,10
3,B,3,180,14
4,B,3,180,14


Obtengo el precio medio por cada grupo:

In [7]:
df_test.groupby("class")["price"].mean().to_frame().reset_index()

,class,price
0,A,12.500000
1,B,12.666667


Con `.agg()` podemos aplicar varias oepraciones:

In [8]:
df_test.groupby("class").agg({'item_id':max,    # Obtengo el maximo valor de item_id
                              'ml':"mean",      # Calculo la media por grupo para ml 
                              'price':sum}).reset_index()    # Caclulo la suma por grupo de price
                        

,class,item_id,price,ml
0,A,2,25,125.000000
1,B,3,38,153.333333


Con `transform()` generar un nuevo valor, dependiendo de alguna característica del grupo.  
Por ej. genero una nueva columna con el precio maximo por clase:

In [9]:
df_test["price_max_class"] = df_test.groupby("class")["price"].transform(max)
df_test

,class,item_id,ml,price,price_max_class
0,A,1,100,10,15
1,A,2,150,15,15
2,B,1,100,10,14
3,B,3,180,14,14
4,B,3,180,14,14


Supongamos que tenemos NAs en la columna _precio_, podemos imputar valores faltantes por grupo.  

In [10]:
# Elimino col. price_max_class
df_test.drop(columns=["price_max_class"], inplace=True)

# Asingo algunos NAs:
df_test.iloc[1,3] = np.nan
df_test.iloc[3,3] = np.nan
df_test

,class,item_id,ml,price
0,A,1,100,10.0
1,A,2,150,NaN
2,B,1,100,10.0
3,B,3,180,NaN
4,B,3,180,14.0


Imputo el precio medio por grupo   
(genero una nueva columna para poder comprar facilmente)

In [11]:
df_test['filled_price'] = df_test.groupby('class')['price'].transform(
    lambda grp: grp.fillna(np.mean(grp))
)
df_test

,class,item_id,ml,price,filled_price
0,A,1,100,10.0,10.0
1,A,2,150,NaN,10.0
2,B,1,100,10.0,10.0
3,B,3,180,NaN,12.0
4,B,3,180,14.0,14.0
